In [1]:
import scanpy as sc
import pandas as pd

tm_droplet_data = sc.read(
    r'../data/tabula_muris/TM_droplet.h5ad',
)
tm_facs_data = sc.read(
    r'../data/tabula_muris/TM_facs.h5ad',
)

In [2]:
tm_droplet_data

AnnData object with n_obs × n_vars = 245389 × 20138
    obs: 'age', 'cell', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'method', 'mouse.id', 'n_genes', 'sex', 'subtissue', 'tissue', 'tissue_free_annotation'
    var: 'n_cells'

In [3]:
# List all tissue types in tm_droplet_data
tm_droplet_data.obs["tissue"].unique()
# List all tissue types in tm_facs_data
tm_facs_data.obs["tissue"].unique()
# List all cell types in tm_droplet_data
tm_droplet_data.obs["cell_ontology_class"].unique()


['keratinocyte', 'basal cell of epidermis', 'Langerhans cell', 'fibroblast of cardiac tissue', 'leukocyte', ..., 'basal epithelial cell of tracheobronchial tree', 'chondrocyte', 'neuroendocrine cell', 'blood cell', 'smooth muscle cell of trachea']
Length: 123
Categories (123, object): ['B cell', 'CD4-positive, alpha-beta T cell', 'CD8-positive, alpha-beta T cell', 'DN3 thymocyte', ..., 'stromal cell', 'thymocyte', 'type II pneumocyte', 'vein endothelial cell']

In [4]:
# Filter only for cells with valid cell ontology class
tm_droplet_data = tm_droplet_data[
    (~tm_droplet_data.obs.cell_ontology_class.isna())
].copy()
tm_facs_data = tm_facs_data[
    (~tm_facs_data.obs.cell_ontology_class.isna())
].copy()

# Add technology labels
tm_droplet_data.obs["tech"] = "10x"
tm_facs_data.obs["tech"] = "SS2"

In [5]:
gene_len = pd.read_csv(
    "https://raw.githubusercontent.com/chenlingantelope/HarmonizationSCANVI/master/data/gene_len.txt",
    delimiter=" ",
    header=None,
    index_col=0,
)
gene_len.head()

,1
0,
0610007C21Rik,94.571429
0610007L01Rik,156.000000
0610007P08Rik,202.272727
0610007P14Rik,104.000000
0610007P22Rik,158.750000


In [6]:
import numpy as np
from scipy import sparse

gene_len = gene_len.reindex(tm_facs_data.var.index).dropna()

tm_facs_data = tm_facs_data[:, gene_len.index].copy() # break the view

gene_len_vec = gene_len[1].values.astype(np.float32)
median_len = np.median(gene_len_vec)

# column‑wise scaling in CSC format
X = tm_facs_data.X.tocsc(copy=True) # -> (n_cells × n_genes)
X = X.multiply(1.0 / gene_len_vec) # divide each column by its length
X = X.multiply(median_len) # multiply by the median length
X.data = np.rint(X.data) # round only the non‑zero entries

tm_facs_data.X = X.tocsr() # store back as CSR (Scanpy’s default)

In [7]:
import scanpy as sc
import pandas as pd

tm_adata = tm_droplet_data.concatenate(tm_facs_data)
tm_adata.layers["counts"] = tm_adata.X.copy()
sc.pp.normalize_total(tm_adata, target_sum=1e4)
sc.pp.log1p(tm_adata)
tm_adata.raw = tm_adata  # keep full dimension safe
sc.pp.highly_variable_genes(
    tm_adata,
    flavor="seurat_v3",
    n_top_genes=2000,
    layer="counts",
    batch_key="tech",
    subset=True,
)

/tmp/ipykernel_3993561/494943004.py:4: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  tm_adata = tm_droplet_data.concatenate(tm_facs_data)


In [8]:
tm_adata

AnnData object with n_obs × n_vars = 356213 × 2000
    obs: 'age', 'cell', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'method', 'mouse.id', 'n_genes', 'sex', 'subtissue', 'tissue', 'tissue_free_annotation', 'tech', 'FACS.selection', 'n_counts', 'batch'
    var: 'n_cells-0', 'n_cells-1', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'log1p', 'hvg'
    layers: 'counts'

In [9]:
celltype_techs = tm_adata.obs.groupby("cell_ontology_class")["tech"].unique()
celltype_status = {}
for celltype, tech_list in celltype_techs.items():
    tech_set = set(tech_list)
    if len(tech_set) == 1:
        if "10x" in tech_set:
            celltype_status[celltype] = "only_10x"
        else:
            celltype_status[celltype] = "only_SS2"
    else:
        celltype_status[celltype] = "both"

# 3) Create a new column in .obs indicating whether a cell's type is only_10x, only_SS2, or both
tm_adata.obs["celltype_tech_availability"] = (
    tm_adata.obs["cell_ontology_class"].map(celltype_status)
)

/tmp/ipykernel_3993561/216714466.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  celltype_techs = tm_adata.obs.groupby("cell_ontology_class")["tech"].unique()


In [11]:
tm_adata.obs['cell_ontology_class'].replace(
    to_replace='pancreatic ductal cel',
    value='pancreatic ductal cell',
    inplace=True
)

/tmp/ipykernel_3993561/1686945273.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tm_adata.obs['cell_ontology_class'].replace(
/tmp/ipykernel_3993561/1686945273.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  tm_adata.obs['cell_ontology_class'].replace(


In [12]:
tm_droplet_data_tissues = tm_droplet_data.obs.tissue.tolist()
tm_droplet_data_tissues = {t for t in tm_droplet_data_tissues}
tm_droplet_data_tissues
print(f'{tm_droplet_data_tissues=}')
print(f'{len(tm_droplet_data_tissues)=}')

tm_facs_data_tissues = tm_facs_data.obs.tissue.tolist()
tm_facs_data_tissues = {t for t in tm_facs_data_tissues}
tm_facs_data_tissues
print(f'{tm_facs_data_tissues=}')
print(f'{len(tm_facs_data_tissues)=}')

tm_all_tissues = tm_droplet_data_tissues.union(tm_facs_data_tissues)
# tm_all_tissues
print(f'{len(tm_all_tissues)=}')

# train_tissues = tm_shared_tissues[:-4]
# test_tissues = tm_shared_tissues[-4:]

# print(f'{train_tissues=}')
# print(f'{test_tissues=}')

# train_tissues=['Large_Intestine', 'Spleen', 'Mammary_Gland', 'Lung', 'Kidney', 'Thymus', 'Bladder', 'Tongue', 'Marrow', 'Trachea']
test_tissues={'Skin', 'Liver', 'Limb_Muscle', 'Pancreas'}
train_tissues = tm_all_tissues.difference(test_tissues) # v3,5_gpu

tm_droplet_data_tissues={'Mammary_Gland', 'Thymus', 'Tongue', 'Skin', 'Fat', 'Trachea', 'Pancreas', 'Marrow', 'Lung', 'Limb_Muscle', 'Large_Intestine', 'Bladder', 'Liver', 'Heart_and_Aorta', 'Kidney', 'Spleen'}
len(tm_droplet_data_tissues)=16
tm_facs_data_tissues={'GAT', 'Skin', 'Kidney', 'Mammary_Gland', 'Marrow', 'Thymus', 'Diaphragm', 'Tongue', 'Trachea', 'Brain_Myeloid', 'BAT', 'Lung', 'MAT', 'Large_Intestine', 'Bladder', 'Aorta', 'SCAT', 'Brain_Non-Myeloid', 'Pancreas', 'Limb_Muscle', 'Heart', 'Liver', 'Spleen'}
len(tm_facs_data_tissues)=23
len(tm_all_tissues)=25


In [13]:
tm_adata_train = tm_adata[
    tm_adata.obs['tissue'].isin(train_tissues)
]
tm_adata_test = tm_adata[
    tm_adata.obs['tissue'].isin(test_tissues)
]

In [ ]:
import pickle
# Create the directory if it doesn't exist
import os
os.makedirs(r'./src/data/tabula_muris/preprocessed', exist_ok=True)

with open(r'./src/data/tabula_muris/preprocessed/tm_adata_train.pkl', 'wb') as f: # NOTE: be careful of where this actually is LOL
    pickle.dump(tm_adata_train, f)

with open(r'./src/data/tabula_muris/preprocessed/tm_adata_test.pkl', 'wb') as f:
    pickle.dump(tm_adata_test, f)

In [15]:
'''import pickle
# Create the directory if it doesn't exist
import os
os.makedirs(r'./src/data/tabula_muris/preprocessed', exist_ok=True)

with open(r'./src/data/tabula_muris/preprocessed/tm_adata_all.pkl', 'wb') as f:
    pickle.dump(tm_adata, f)
    '''

"import pickle\n# Create the directory if it doesn't exist\nimport os\nos.makedirs(r'./src/data/tabula_muris/preprocessed', exist_ok=True)\n\nwith open(r'./src/data/tabula_muris/preprocessed/tm_adata_all.pkl', 'wb') as f:\n    pickle.dump(tm_adata, f)\n    "